## Implementation

In [ ]:
#| default_exp server

In [ ]:
#| export
import os
from typing import List, Tuple, Callable, Awaitable, Dict
from langchain.vectorstores.base import VectorStore
from langchain.schema.runnable import RunnableSequence
from langchain.chat_models.base import BaseChatModel
from langchain.embeddings.base import Embeddings
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from pydantic.dataclasses import dataclass
from datetime import datetime
import tiktoken
from pino_inferior.core import SERVER_OPENAI_API_KEYS, \
    OPENAI_FALLACY_MODEL, OPENAI_AGENT_MODEL, OPENAI_CONTEXT_MODEL, \
    SERVER_MAX_FALLACIES_LENGTH, SERVER_MAX_THREAD_LENGTH, \
    SERVER_MAX_CONTEXT_LENGTH, \
    SERVER_AGENT_MAX_ITERATIONS, \
    SERVER_MAX_CONTEXT_EXTRACTOR_POST_LENGTH, \
    SERVER_HOST, SERVER_PORT, \
    VECTOR_DB, VECTOR_DB_PARAMS, MEMORY_PARAMS, \
    OPENAI_MEMORY_EMBEDDER_MODEL, \
    read_file
from pino_inferior.fallacy import build_fallacy_detection_chain, read_fallacies, \
    FALLACIES_FNAME
from pino_inferior.models import aengine
from pino_inferior.memory import Memory
from pino_inferior.agent import RolePlayAgent, ToolDescription, \
    TOOLS_PROMPTS_DIR, \
    INPUT_FALLACY_QUERY, OUTPUT_FALLACY_QUERY, \
    INPUT_RETRIEVER_QUERY, OUTPUT_RETRIEVER_DOCUMENTS, \
    FallacyLengthConfig, \
    LengthConfig as AgentLengthConfig, \
    PromptMarkupConfig as AgentPromptMarkupConfig, \
    Message as AgentMessage, \
    AGENT_INPUT_TIME, AGENT_INPUT_CONTEXT, AGENT_INPUT_FALLACIES, \
    AGENT_INPUT_HISTORY, AGENT_INPUT_TOOLS, AGENT_INPUT_USERNAME, \
    AGENT_INPUT_CHARACTER, AGENT_INPUT_GOAL, AGENT_INPUT_STYLE_EXAMPLES, \
    AGENT_INPUT_STYLE_DESCRIPTION
from langchain_openai_limiter import LimitAwaitChatOpenAI, ChooseKeyChatOpenAI, \
    LimitAwaitOpenAIEmbeddings, ChooseKeyOpenAIEmbeddings
from pino_inferior.function_callbacks import LLMEventType, AsyncLLMCallback, \
    AsyncFunctionalStyleChatCompletionHandler
from pino_inferior.context_extractor import build_context_extractor_chain, \
    LengthConfig as ContextExtractorLengthConfig, \
    PromptMarkupConfig as ContextExtractorPromptMarkupConfig, \
    CONTEXT_INPUT_TEXT, CONTEXT_INPUT_POST_TIME, \
    CONTEXT_INPUT_GOALS, CONTEXT_INPUT_CURRENT_TIME, \
    CONTEXT_INPUT_USERNAME, CONTEXT_INPUT_CHARACTER, \
    CONTEXT_OUTPUT_CONTEXT
import traceback
from copy import copy
import pandas as pd
from sqlalchemy.ext.asyncio import AsyncSession
import asyncio
from websockets.server import serve, WebSocketServerProtocol
from pydantic.tools import parse_obj_as
import json
from pydantic.json import pydantic_encoder

In [ ]:
#| export
try:
    __file__
    IS_JUPYTER = False
except NameError:
    IS_JUPYTER = True

### OpenAI wrappers

In [ ]:
#| export
def _initialize_openai_chat_model(name: str, callbacks: List[AsyncFunctionalStyleChatCompletionHandler]) -> Tuple[ChatOpenAI, BaseChatModel]:
    gpt = ChatOpenAI(
        model_name=name,
        streaming=True,
    )
    limited_gpt = LimitAwaitChatOpenAI(chat_openai=gpt)
    choose_key_gpt = ChooseKeyChatOpenAI(openai_api_keys=SERVER_OPENAI_API_KEYS,
                                         chat_openai=limited_gpt,
                                         callbacks=callbacks,)
    return gpt, choose_key_gpt

In [ ]:
#| export
def _initialize_openai_embedder_model(name: str) -> Tuple[OpenAIEmbeddings, Embeddings]:
    embedder = OpenAIEmbeddings(
        model=name,
    )
    limited_embedder = LimitAwaitOpenAIEmbeddings(openai_embeddings=embedder)
    choose_key_embedder = ChooseKeyOpenAIEmbeddings(openai_api_keys=SERVER_OPENAI_API_KEYS,
                                                    openai_embeddings=limited_embedder)
    return embedder, choose_key_embedder

In [ ]:
#| export
def _parse_json_as(cls, json_text):
    return parse_obj_as(cls, json.loads(json_text))

### Common parts of query processing

In [ ]:
#| export
@dataclass
class UserDescription:
    name: str
    character: str
    goals: str


@dataclass
class UserDescriptionWithStyle(UserDescription):
    style_examples: List[str]
    style_description: str

In [ ]:
#| export
RequestId = int


@dataclass
class Request:
    id: RequestId

In [ ]:
#| export
CallbackSystem = str
CallbackType = str
CallbackTime = datetime
CallbackResponse = str

AsyncCallback = Callable[[RequestId, CallbackSystem, CallbackType, CallbackTime, CallbackResponse], Awaitable[None]]

### Individual API requests

#### Roleplay agent API

In [ ]:
#| export
FALLACY_TOOL_DESCRIPTION = read_file(os.path.join(TOOLS_PROMPTS_DIR, "fallacy.txt"))
MEMORY_TOOL_DESCRIPTION = read_file(os.path.join(TOOLS_PROMPTS_DIR, "memory.txt"))
FALLACIES = read_fallacies(FALLACIES_FNAME)

In [ ]:
#| export
@dataclass
class Message:
    author: str
    time: str
    content: str


@dataclass
class CommentRequest(Request):
    context: str
    history: List[Message]
    user: UserDescriptionWithStyle

In [ ]:
#| export
def _initialize_fallacy_tool(llm: ChatOpenAI) -> Tuple[ToolDescription, RunnableSequence]:
    encoding = tiktoken.encoding_for_model(llm.model_name)
    length_config = FallacyLengthConfig(
        length_function=lambda text: len(encoding.encode(text)),
        max_messages_length=SERVER_MAX_THREAD_LENGTH,
        max_fallacies_length=SERVER_MAX_FALLACIES_LENGTH,
    )
    description = ToolDescription(
        name="fallacy",
        description=FALLACY_TOOL_DESCRIPTION,
        input_key=INPUT_FALLACY_QUERY,
        output_key=OUTPUT_FALLACY_QUERY,
    )
    chain = build_fallacy_detection_chain(llm, length_config)
    return description, chain

In [ ]:
#| export
def _initialize_memory_tool(embedder: OpenAIEmbeddings) \
    -> Tuple[ToolDescription, RunnableSequence]:
    vector_store = VECTOR_DB(embedding_function=embedder, **VECTOR_DB_PARAMS)
    container = Memory(
        engine=aengine,
        vector_db=vector_store,
        **MEMORY_PARAMS
    )
    description = ToolDescription(
        name="memory",
        description=MEMORY_TOOL_DESCRIPTION,
        input_key=INPUT_RETRIEVER_QUERY,
        output_key=OUTPUT_RETRIEVER_DOCUMENTS,
    )
    chain = container.build_retriever_chain()
    return description, chain

In [ ]:
#| export
def _initialize_agent(fallacy_callbacks: List[AsyncFunctionalStyleChatCompletionHandler],
                      agent_callbacks: List[AsyncFunctionalStyleChatCompletionHandler]) \
                         -> Tuple[ChatOpenAI, ChatOpenAI, OpenAIEmbeddings, RolePlayAgent]:
    fallacy_gpt, fallacy_llm = _initialize_openai_chat_model(OPENAI_FALLACY_MODEL, fallacy_callbacks)
    agent_gpt, agent_llm = _initialize_openai_chat_model(OPENAI_AGENT_MODEL, agent_callbacks)
    embeddings_openai, embeddings = _initialize_openai_embedder_model(OPENAI_MEMORY_EMBEDDER_MODEL)
    agent_gpt_encoding = tiktoken.encoding_for_model(agent_gpt.model_name)
    
    fallacy_tool = _initialize_fallacy_tool(fallacy_llm)
    memory_tool = _initialize_memory_tool(embedder=embeddings)
    tools = [fallacy_tool, memory_tool]

    agent = RolePlayAgent(
        tools=tools,
        lengths=AgentLengthConfig(
            cut_function=lambda text, length: agent_gpt_encoding.decode(agent_gpt_encoding.encode(text)[:length]),
            length_function=lambda text: len(agent_gpt_encoding.encode(text)),
            max_messages_length=SERVER_MAX_THREAD_LENGTH,
            max_context_length=SERVER_MAX_CONTEXT_LENGTH,
        ),
        prompt_markup=AgentPromptMarkupConfig(),
        llm=agent_llm,
        max_iter=SERVER_AGENT_MAX_ITERATIONS,
    )
    
    return fallacy_gpt, agent_gpt, embeddings_openai, agent

In [ ]:
#| export
async def process_comment_request(request: CommentRequest, callback: AsyncCallback) -> None:
    async def _inner_callback(system: str, envent_type: str, time: datetime, content: str) -> None:
        await callback(request.id, system, envent_type, time, content)

    async def _fallacy_callback(event_type: LLMEventType, time: datetime, content: str) -> None:
        await _inner_callback("fallacy", event_type.value, time, content)

    async def _agent_callback(event_type: LLMEventType, time: datetime, content: str) -> None:
        await _inner_callback("agent", event_type.value, time, content)

    await _inner_callback("system", "START", datetime.now(), "")
    try:
        fallacy_gpt, agent_gpt, _, agent = _initialize_agent(
            [AsyncFunctionalStyleChatCompletionHandler(_fallacy_callback)],
            [AsyncFunctionalStyleChatCompletionHandler(_agent_callback)]
        )
        inputs = {
            AGENT_INPUT_TIME: datetime.now(),
            AGENT_INPUT_CONTEXT: request.context,
            AGENT_INPUT_FALLACIES: FALLACIES,
            AGENT_INPUT_HISTORY: [
                AgentMessage(message.author, pd.to_datetime(message.time), message.content)
                for message in request.history
            ],
            AGENT_INPUT_TOOLS: agent.tools,
            AGENT_INPUT_USERNAME: request.user.name,
            AGENT_INPUT_CHARACTER: request.user.character,
            AGENT_INPUT_GOAL: request.user.goals,
            AGENT_INPUT_STYLE_EXAMPLES: request.user.style_examples,
            AGENT_INPUT_STYLE_DESCRIPTION: request.user.style_description,
        }
        response = await agent.arun(inputs)
        await _inner_callback("system", "END", datetime.now(), response)
    except Exception as err:
        await _inner_callback("system", "ERROR", datetime.now(), traceback.format_exception(err))
        raise err

In [ ]:
#| export
async def aprint(*args, **kwargs):
    print(*args, **kwargs)

In [ ]:
await process_comment_request(
    CommentRequest(
        id=1,
        context="Пост о войне России и Украины",
        history=[
            Message(
                "Moonlight",
                "2023-10-01 11:29:00",
                "Мы скоро закончим с Украиной."
            ),
            Message(
                "alex4321",
                "2023-10-01 11:30:00",
                "А что, случился какой-то прогресс после 6 месяцев взятия Бахмута?\n\n" + \
                    "Ну, чтобы подозревать что это произойдёт вскоре, " + \
                    "а не затянется на годы независимо от исхода."
            ),
            Message(
                "Moonlight",
                "2023-10-01 11:31:00",
                "Время - ресурс, у нас его дохуя."
            )
        ],
        user=UserDescriptionWithStyle(
            name="alex4321",
            character="you are a programmer, 29 y.o. male",
            goals="Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions. And that Russia should quit it.",
            style_description="- Non-formal style, using mainly Russian language " + \
                "(my English is a bit screwed up)" + \
                "\n- Brief. Most time.\n" + \
                "- Overuse memes sometimes.",
            style_examples=[
                "В вакууме, да :Yoba:. Оба же тут существовали всё время или имели таки достигнутое соглашение, чтобы было от чего отталкиваться как опорной точки :Yoba:",
                "Не особо-то может. Не привлекая население в виде не 1% принудительного мармелада и пары процентов добровольного, а в виде процентов 10.\nА то, чтобы привлечь большое количество - неплохо бы, чтобы они понимали, нахуя это им надо. А то так численность военкомов может начать неприемлемо быстро падать, а следом их желание работать.\nА с этим у пропаганды проблема. Вот с чем у них нет проблем, так это с стимуляцией пассивности, но это обратно нужному (для названной вами задачи).\nДа и опять же - ну вот убедишь ты в идее не какого-нибудь Стрелкова и клуб рассерженных долбоёбов, а большое число людей. Что делать, когда (не если, а когда) идея станет неактуальной? Показательной посадкой пары человек дело не закончится же.\n",
                "Точнее не так - смену она не устраивала.\nОна просто выстрелила себе в ногу так, что потом что-то новое приходилось строить не апгрейдом предыдущей системы, а из кусков её трупа.",
            ],
        )
    ),
    aprint,
)

1 system START 2023-11-08 04:57:51.822090 
1 agent LLM_START 2023-11-08 04:57:52.829088 system: Roleplay like you're a human, not an AI model.

human: ## Character
It's 08 Nov 2023 04:57 now.

Imagine you're alex4321 - you are a programmer, 29 y.o. male

Your goals are:
```
Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions. And that Russia should quit it.
```

## Character text style
A required response style description:
```
- Non-formal style, using mainly Russian language (my English is a bit screwed up)
- Brief. Most time.
- Overuse memes sometimes.
```
A few examples of messages to make you mimic the style:
```
В вакууме, да :Yoba:. Оба же тут существовали всё время или имели таки достигнутое соглашение, чтобы было от чего отталкиваться как опорной точки :Yob

Error in AsyncFunctionalStyleChatCompletionHandler.on_llm_error callback: KeyError('x-ratelimit-limit-requests')


1 agent LLM_ERROR 2023-11-08 04:57:53.590782 Traceback (most recent call last):
  File "c:\Users\alex4321\anaconda3\envs\pino-inferior\Lib\asyncio\tasks.py", line 267, in __step
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\alex4321\anaconda3\envs\pino-inferior\Lib\site-packages\langchain\chat_models\base.py", line 532, in _agenerate_with_cache
    return await self._agenerate(
           ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\alex4321\anaconda3\envs\pino-inferior\Lib\site-packages\langchain_openai_limiter\choose_key_chat_openai.py", line 119, in _agenerate
    return await chat_openai._agenerate(messages,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\alex4321\anaconda3\envs\pino-inferior\Lib\site-packages\langchain_openai_limiter\limit_await_chat_openai.py", line 134, in _agenerate
    return await self.chat_openai._agenerate(messages, stop, run_manager, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyError: 'x-ratelimit-limit-requests'

#### Context parser API

In [ ]:
#| export
@dataclass
class ContextRequest(Request):
    text: str
    time: str
    user: UserDescription

In [ ]:
#| export
def _initialize_context_extractor(callbacks: List[AsyncFunctionalStyleChatCompletionHandler]) -> RunnableSequence:
    _, llm = _initialize_openai_chat_model(OPENAI_CONTEXT_MODEL, callbacks)
    encoding = tiktoken.encoding_for_model(llm.model_name)
    context_extractor = build_context_extractor_chain(
        llm,
        lengths=ContextExtractorLengthConfig(
            cut_function=lambda text, length: encoding.decode(encoding.encode(text)[:length]),
            length_function=lambda text: len(encoding.encode(text)),
            max_post_length=SERVER_MAX_CONTEXT_EXTRACTOR_POST_LENGTH,
            max_response_length=SERVER_MAX_CONTEXT_LENGTH,
        ),
        prompts=ContextExtractorPromptMarkupConfig()
    )
    return context_extractor

In [ ]:
#| export
async def process_context_extraction_request(request: ContextRequest, callback: AsyncCallback) -> None:
    async def _inner_callback(system: str, envent_type: str, time: datetime, content: str) -> None:
        await callback(request.id, system, envent_type, time, content)
    
    async def _context_callback(event_type: LLMEventType, time: datetime, content: str) -> None:
        await _inner_callback("context", event_type.value, time, content)
    
    await _inner_callback("system", "START", datetime.now(), "")
    try:
        context_extractor = _initialize_context_extractor([
            AsyncFunctionalStyleChatCompletionHandler(_context_callback)
        ])
        response = await context_extractor.ainvoke({
            CONTEXT_INPUT_TEXT: request.text,
            CONTEXT_INPUT_POST_TIME: pd.to_datetime(request.time),
            CONTEXT_INPUT_GOALS: request.user.goals,
            CONTEXT_INPUT_CURRENT_TIME: datetime.now(),
            CONTEXT_INPUT_USERNAME: request.user.name,
            CONTEXT_INPUT_CHARACTER: request.user.character,
        })
        response_text = response[CONTEXT_OUTPUT_CONTEXT]
        await _inner_callback("system", "END", datetime.now(), response_text)
    except Exception as err:
        await _inner_callback("system", "ERROR", datetime.now(), traceback.format_exception(err))
        raise err

In [ ]:
TEXT = """Будет ли мобилизация осенью?⁠⁠
Самый, наверное, волнующий россиян вопрос на сегодняшний день, обросший обильными слухами вплоть до вброса конкретных дат. Хочу его разобрать и озвучить свои соображения на тему, будет ли мобилизация, если да – то когда.

Мой канал в ТГ: https://t.me/artjockey

Напомню старым и новым читателям, что у меня нет никаких инсайдов, я просто озвучу свое мнение и постараюсь его обосновать. С чем можно согласиться или не согласиться.

Будет ли мобилизация осенью? Политика, Спецоперация, Война на Украине, Министерство обороны, Мобилизация, Война, Частичная мобилизация, Россия, Призыв, Длиннопост
Первая волна мобилизации
Она началась прошлой осенью и с 21 сентября по 31 октября было призвано 318 тысяч официально. При этом, указ о мобилизации юридически продолжает действовать. То есть, формально, новую волну власть может не объявлять, а просто, опираясь на действующий указ, начать новый призыв.

Для чего нужна была мобилизация? Численность группировки вторжения ВС РФ была около 200 тысяч человек (с учетом войск еще независимых ДЛНР). Украинская армия мирного времени – примерно 260 тысяч человек. То есть, силы были примерно равны, даже в пользу ВС РФ, потому что у ВСУ это не вся сухопутная армия, а с учетом всех тыловых служб, штаба, пограничников и т.д.

Но сразу после начала войны, украинское правительство объявляет всеобщую мобилизацию, вполне логичный и правильный шаг. Численность украинской армии быстро увеличивается и к октябрю достигает 700 тысяч человек, по словам украинского министра обороны. Конечно, далеко не все эти люди находились на передовой, то есть, не являлись «штыками». Но перекос в численности воюющих армий уже очевиден и было понятно, что продолжать завоевательную войну имея в 2-3 раза меньше людей невозможно.

Частичная мобилизация позволила ВС РФ достичь примерного паритета с ВСУ по численности и продолжить войну в условно равном соотношении сил. Даже если перекос в чью-то сторону сохранялся, он был уже не в разы, а в десятки процентов, не более. Это позволило и равномерно укрепить фронт, чтобы не допустить повторения сценария наступления ВСУ на Изюм.

Но всеобщая мобилизация на территории Украины не прекратилась, украинская армия продолжила пополняться, зимой стали известны и планы Киева провести наступление в 2023-м году. В таких условиях уже было понятно, что вторая волна – неизбежна. ВС РФ необходимо, хотя бы, поддерживать паритет по численности. Многие источники писали, что мобилизация будет проведена в начале года, чтобы иметь обученные резервы к концу весны-лету. Я тоже думал, что мобилизацию объявят.

Стратегия Кремля
Как мы знаем, вторую волну мобилизации не объявили и связано это с курсом руководства России на изоляцию войны от основной массы российского общества. Я не сомневаюсь, что подготовка действительно велась, но в какой-то момент планы поменялись.

Для среднего россиянина война идет и, по этой стратегии, должна идти где-то в телевизоре, потому и не война, а СВО. Да, есть некий процент людей, кого затронула мобилизация в первую волну, но их, относительно общего населения страны, немного.

Также временно вопрос пополнения закрыл собой Вагнер, через который, как мы теперь знаем, прошло 78 тысяч человек, что довольно много. Это где-то 25% от частично мобилизованных.

Но общая проблема с поддержкой паритета по численности личного состава никуда не делась и ее нужно было как-то решать. Решение было озвучено весной, вместо второй волны мобилизации предполагается до конца года набрать 400 тысяч людей на контрактную службу. Вот эти новые контрактники и должны заменить собой «вторую волну».

Скрытая мобилизация
Есть такое понятие, как скрытая мобилизация. То есть, набор в войска не обязательно ведется с официальными объявлениями по телевизору, фанфарами и всесторонним освещением в СМИ.

При этом, не стоит понимать термин настолько буквально, что это некие тайные мероприятия, о которых никто не должен узнать. Совсем не обязательно. Это просто некий общий комплекс мер, призванный замаскировать или скрыть отдельные детали процесса или же прикрыть его юридически. Так, например, официальная мобилизация государства может являться поводом для объявления войны, а, скажем, увеличение численности пограничных войск и парамилитарных образований – таким поводом уже не будет.

Также, скрытая мобилизация может скрывать не сам процесс, а его детали. При частичной мобилизации все знали и сколько людей будет призвано, и куда они будут направлены. При скрытой мобилизации такие подробности могут не озвучиваться, хотя сам проходящий процесс и является достоянием общественности.

Вот нынешняя стратегия российского руководства и является проведением скрытой мобилизацией. Официально, контрактники набираются не для участия в СВО, а согласно стратегическому плану развития армии, которое не так давно было подписано Путиным и предусматривает увеличение численности ВС РФ и вне контекста войны.

Методы скрытой мобилизации
Сейчас в РФ задействовано три основных источника пополнения личного состава.

Во-первых, это прямой поток добровольцев, заключающих контракты на службу в рядах армии. Жители России, думаю, видели и агрессивную рекламу службы, кто-то сталкивался с агитаторами на работе, а кто-то даже получал повестку для уточнения данных в военкомат, где его уговаривали заключить контракт.

Во-вторых, это перевод на контракт срочников. Каждый срочник может заключить контракт, который пойдет в зачет срочки.

В-третьих, это продолжающийся набор в тюрьмах, в основном, в отряды Шторм вместо Вагнера.

Все эти три метода позволяют увеличить численность личного состава в рядах вооруженных сил, и они все задействованы. Но есть еще один метод, это уже мобилизованные, которым предлагают заключить контракт. Таки люди тоже идут в статистику, но вот на общее число солдат они не влияют.

Есть и менее очевидные способы, вот новость прям с Пикабу. Когда в ходе недавних массовых рейдов среди мест компактного проживания граждан, недавно получивших паспорт, выявляли тех, кто еще не стал на учет в военкомат и помогали им это сделать прям с доставкой к месту оформления документов.

Успехи скрытой мобилизации
Вот теперь о цифрах, как ни странно, они нам известны. Бывший президент, Дмитрий Медведев, который как раз и курирует этот процесс, озвучил, что с 1 апреля по август на контракт записалась 231 тысяча человек.

Это довольно много и почти догоняет общее число мобилизованных в первую волну. То есть, по факту, перекрывает потребности во второй волне мобилизации. И да, наверняка, люди сразу скажут о вот тех самых приписках, когда уже мобилизованных переводят на контракт, учитывают в статистике, но фактическое положение дел не меняется. К сожалению, мы не знаем, сколько таких людей среди этих 231 тысячи есть.

Но у нас есть взгляд и с другой стороны. Скрытую мобилизацию в РФ совсем недавно комментировал представитель ГУР Украины, он подтвердил, что «разными методами» в месяц мобилизуется 20 тысяч человек. То есть, даже с учетом всех манипуляций, поток новобранцев в зону СВО достаточно большой, в течение года это и будут те самые примерно 300 тысяч человек, которые были в первой волне.

Промежуточные выводы
Вторая волна мобилизации в России уже идет и идет достаточно успешно. Даже при мобилизации 20 тысяч человек в месяц, это в общем позволяет руководству РФ отказаться от проведения отдельной второй волны частичной мобилизации и закрыть все потребности за счет добровольцев (условно, все мы понимаем, что часть людей заключает контракт добровольно-принудительно).

Также вспоминаем, что я написал в начале – указ о частичной мобилизации юридически еще действует. Можно предположить, что при недостаточном потоке новобранцев будет проведена не вторая волна по образу первой, а еще включен еще один механизм. Начнут рассылать повестки ежемесячно в небольшом количестве, уже прямо призывая не 300 тысяч человек за раз, а, скажем, по 10 тысяч человек в месяц.

Именно так происходит на Украине. Процесс рассылки и раздачи повесток идет постоянно без каких-либо волн. Повестка – это вызов на «уточнение данных», где уже вместо нее вручают мобилизационное предписание, по украинской практике, даже не выпуская из военкомата. И напомню, что в России рассылаются похожие повестки и сейчас, только с целью уговорить заключить контракт, но при необходимости, просто начнут сразу давать предписания.

Юридические изменения
Между тем, Госдума ввела ряд изменений в законодательство, которые тоже серьезно всколыхнули общество. Во-первых, это увеличение возраста запаса, во-вторых, увеличение возраста призыва на срочную службу, в-третьих, штрафы для уклонистов.

Из этих изменений делается вывод, что мобилизация готовится осенью. Но это не совсем так, первые два пункта точно никак не могут повлиять на осеннюю мобилизацию. Возраст учета вообще вводится поэтапно, каждый год на один год. А возраст призыва увеличивается с 2024-го, но не с 2023-го года. На мой взгляд, это не подготовка к СВО вообще, а подготовка к вероятной эскалации конфликта. Я уже как-то говорил, что мы, возможно, находимся в 1938-м или в 1913-м году, просто этого еще не знаем. А лет через 20 будем спорить, является ли Российско-Украинская война отдельным конфликтом или это один из театров действий Третьей мировой.

Остается третий пункт, ужесточение штрафов для уклонистов. Кто-то это воспринял как легитимизацию взятки за уклонение, но ведь штраф не освобождает от обязанности явиться в военкомат по повестке.

Я считаю, что тут смысл в другом. Чуть выше я писал, что один из источников пополнения личного состава в зоне СВО – это срочники. И здесь тоже есть изменения, срочник сможет заключить контракт через месяц после призыва, а раньше мог только через три.

И вот эти все изменения направлены на то, чтобы упростить осенний призыв на срочную служба, а уже на срочке начнут обрабатывать призывников на тему подписания контракта. И здесь, конечно, к кому-то могут применить кнут, кто-то поведется на пряник в виде, в первую очередь, зарплаты. Штраф этому только поможет, ведь штрафник может записаться на контракт и с нескольких зарплат с ним расплатиться.

Есть и еще одно не самое заметное изменение. С недавнего времени, иностранцы могут заключить контракт с ВС РФ на один год, а затем получить гражданство по упрощенной процедуре.

Логические «против»
Кроме того, что реальные и юридические действия руководства России нацелены на то, чтобы избежать проведения второй волны частичной мобилизации, есть еще и объективные причины, почему ее не стоит проводить, во всяком случае, сейчас.

Проведений первой волны выявило ряд проблем, которые такой метод создает. И здесь даже можно обойтись без пруфов, просто само по себе очевидно, что одномоментный призыв в армию 300 тысяч человек за 1,5 месяца сильно перегружает все тыловые службы. Так, например, призыв на срочку – это 120-150 тысяч человек в течение 3-х месяцев.

При этом, на Украине, например, при всеобщей мобилизации отменили срочную службу, но в России при частичной – нет. И в прошлом году это привело к тому, что срок призыва на срочку пришлось сдвинуть на месяц.

Зима и поздняя осень – это неудобное время для подготовки личного состава. В отличие от срочников, которые, условно, будут «красить траву», эти люди отправляются на войну и нужно постараться их хоть как-то обучить. Зимой это не слишком удобно из-за погоды, температуры и всего остального. Если на полигоне грязь, в которой танк застрянет, обучения не получится.

Конечно, возникает правильный вопрос, если все эти проблемы очевидны даже с дивана, зачем тогда проводилась первая волна? Она была продиктована военной необходимостью, когда с ней очень долго тянули, а потом вынуждены были добирать людей на войну в условиях, что они нужны были еще на вчера (на начало сентября в Харьковской области).

Сейчас же ситуация на фронте явно не диктует срочной необходимости добирать людей. В моем прошлом посте я показал, что Россия проводит успешную оборонительную операцию, нет признаков обрушения фронта или возможных скорых успехов ВСУ. При том, у России хватает сил не только обороняться, но еще и наступать на другом фланге на Купянск.

Вопрос ротации
Еще один часто поднимаемый вопрос, это то, что мобилизация необходима для ротации, а то и для демобилизации частично мобилизованных. Потому что они уже почти год как воюют, а хотят по домам. Ну или хотя бы на ротацию.

Возможно, меня поправят действующие военные, но у меня сложилось мнение, что все эти нормативы по ротации, записанные в уставах, в нынешней войне пошли к одному месту с обеих сторон. В реальности, классических ротаций войск не проводит ни Россия, ни Украина, за исключением частей, понесших сильные потери. Для отдыха военнослужащим дают отпуска, я знаю лично мобилизованных, кто уже успел побывать дома, а кто-то и два раза каким-то образом, но само подразделение в тыл не отводится.

Что же касается демобилизации – ее не планируется точно и про это говорил Путин в июне на встрече с военкорами. Ему задавали этот вопрос, от ответа он ушел, сказав, что все будет зависеть от ситуации на фронте. Потому рассчитывать на демобилизацию через год я бы точно не стал. Возможно, такие процессы начнутся к концу второго года службы мобилизованных, но точно не в ближайшее время и Путин ясно дал это понять.

К тому же, среди мобилизованных активно идет агитация подписывать контракт, который на время СВО является бессрочным. Вот, возможно, как раз к концу второго года, когда уже и будут более точные расклады по ходу боевых действий с учетом вероятного наступления ВС РФ, начнут отпускать. Тогда получится, что кто-то выбыл из-за потерь, кто-то подписал контракт и никуда не денется, а вот оставшихся хотя бы частично можно будет демобилизовать. Вероятно, по каким-то критериям, в первую очередь, тех, кто постарше, у кого много детей и т.д. И при условии, что не будет эскалации конфликта с военным положением и всеобщей мобилизацией, конечно.

Общие выводы и советы
На мой взгляд, ничего не указывает на то, что в России готовится вторая волна мобилизации осенью, как пишут некоторые медийные источники. Наоборот, принимаются меры, чтобы этой волны избежать. В реальности, мобилизация уже идет, только совсем другими методами, не теми, что были задействованы прошлой осенью.

При этом, напомню, что у меня нет инсайдов и я не принимаю решения, за чужую глупость я не в ответе. Может быть, я не прав и российское руководство решит зачем-то еще раз прыгнуть на мобилизационные грабли, но вроде я постарался обосновать, почему этого не должны сделать.

В реальности же, если в МО РФ видят, что поток контрактников недостаточен для закрытия потребностей в живой силе, я думаю, что будет задействован механизм настоящей скрытой мобилизации – рассылка повесток ежемесячно без широкой огласки и без резких «волн», чтобы не напрягать армейский тыл и максимально скрывать это от общества.

То есть, если лично вы опасаетесь быть мобилизованным, я бы дал совет не ждать повторения событий прошлого года, а внимательно следить за новостями. Конечно же, не федеральными, а местными, особенно, неформальными СМИ типа групп в Телеграме и ВК, где выкладывают локальные новости. Слушать, что говорит окружение, коллеги и даже читать оппозиционные СМИ, они точно напишут. И обращать внимание не на сам факт рассылки повесток, их постоянно рассылают для сверки данных, а вот на случаи, когда человек пошел, а ему там мобилизационное предписание выписали. Вот когда такое заметите – вот тогда собирайтесь на дачу или в Верхний Ларс, куда запланировано.

В целом же, некая масштабная мобилизация даже в виде волн – возможна, но для нее должны сложиться объективные обстоятельства непосредственно на фронте. И ясны они будут ближе к лету 2024-го года.

Мой канал в ТГ, в нем выходит намного больше разных постов, никаких репостов, исключительно мои авторские тексты: https://t.me/artjockey


#[моё] #Политика #Спецоперация #Война на Украине #Министерство обороны #Мобилизация #Война #Частичная мобилизация #Россия #Призыв #Длиннопост"""

In [ ]:
await process_context_extraction_request(
    ContextRequest(
        id=2,
        text=TEXT,
        time="2023-08-01 11:00:12",
        user=UserDescription(
            name="alex4321",
            character="you are a programmer, 29 y.o. male",
            goals="Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions. And that Russia should quit it.",
        )
    ),
    aprint,
)

In [ ]:
## TODO: Add remember request.

### Server

In [ ]:
#| export
ApiMethodImplementation = Callable[[Request, AsyncCallback], Awaitable[None]]
ApiMethods = Dict[str, Tuple[ApiMethodImplementation, type]]

In [ ]:
#| export
def _parse_message(methods: ApiMethods, message: str) -> Tuple[ApiMethodImplementation, Request]:
    method, params = message.split(" ", maxsplit=1)
    assert method in methods
    method_implementation, request_class = methods[method]
    request = _parse_json_as(request_class, params)
    return method_implementation, request    

In [ ]:
_parse_message(
    {
        "comment": (process_comment_request, CommentRequest),
        "context": (process_context_extraction_request, ContextRequest),
    },
    "context " + json.dumps(ContextRequest(
        id=2,
        text=TEXT,
        time="2023-08-01 11:00:12",
        user=UserDescription(
            name="alex4321",
            character="you are a programmer, 29 y.o. male",
            goals="Convince as much people as possible that the ongoing war between Russia and Ukraine is only bringing more problem, that it do not solve any issues, and moreover - break the impression that it is widely supported. To do so - express your opinion, and debunk opponents opinions. And that Russia should quit it.",
        )
    ), default=pydantic_encoder)
)

In [ ]:
#| export
async def server() -> None:
    methods = {
        "comment": (process_comment_request, CommentRequest),
        "context": (process_context_extraction_request, ContextRequest),
    }

    async def process(websocket: WebSocketServerProtocol) -> None:
        async def _send(id: RequestId,
                  callback_system: CallbackSystem,
                  callback_type: CallbackType,
                  time: CallbackTime,
                  response: CallbackResponse) -> None:
            await websocket.send(json.dumps({
                "id": id,
                "callbackSystem": callback_system,
                "callbackType": callback_type,
                "time": str(time),
                "response": response
            }))

        message: str
        # TODO: parallel
        async for message in websocket:
            handler, request = None, None
            try:
                handler, request = _parse_message(methods, message)
            except Exception as err:
                await _send(-1, "system", "ERROR", datetime.now(), f"Can't parse request: {traceback.format_exception(err)}")
            if handler is not None and request is not None:
                asyncio.create_task(handler(request, _send))
    
    async with serve(process, SERVER_HOST, SERVER_PORT):
        await asyncio.Future()  # run forever

In [ ]:
#| export
if (__name__ == "__main__") and (not IS_JUPYTER):
    asyncio.run(server())

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()